In [ ]:
#import sys
#sys.path.insert(0, '/global/common/software/lsst/common/miniconda/current/lib/python3.6/site-packages')
#sys.path.insert(0, '/path/to/your/cloned/gcr-catalogs')

In [ ]:
import numpy as np
from GCR import GCRQuery
import GCRCatalogs

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
catalog = GCRCatalogs.load_catalog('dc2_coadd_run1.1p')

In [ ]:
sorted(catalog.list_all_quantities())

In [ ]:
catalog.available_tracts_and_patches[:20]

In [ ]:
catalog.available_tracts

In [ ]:
tract_number = 4850
for d in catalog.get_quantities(['ra', 'dec'], native_filters=[(lambda x: x==tract_number, 'tract')], return_iterator=True):
    plt.scatter(d['ra'], d['dec'], s=2);
plt.xlabel('RA');
plt.ylabel('Dec');
plt.title('Tract {}'.format(tract_number));

In [ ]:
tract_number = 4850
mags = ['mag_g_cModel', 'mag_r_cModel', 'mag_i_cModel']
d = catalog.get_quantities(mags, 
                           filters=[GCRQuery((np.isfinite, key)) for key in mags] + ['mag_r_cModel < 25'], 
                           native_filters=[(lambda x: x==tract_number, 'tract')])
c = plt.scatter(d['mag_g_cModel'] - d['mag_r_cModel'], d['mag_r_cModel'] - d['mag_i_cModel'], s=2, c=d['mag_r_cModel']);
plt.colorbar(c, label='$r$')
plt.xlabel('$g-r$');
plt.ylabel('$r-i$');


Here's an example from https://github.com/LSSTDESC/DC2_Repo/blob/issues/155/Validation/lensing_distributions.ipynb

In [ ]:
catalog.add_quantity_modifier('HSM_res', 
                              'ext_shapeHSM_HsmShapeRegauss_resolution', 
                              overwrite=True)
catalog.add_quantity_modifier('HSM_ell', 
                              (np.hypot, 'ext_shapeHSM_HsmShapeRegauss_e1', 'ext_shapeHSM_HsmShapeRegauss_e2'), 
                              overwrite=True)

In [ ]:
# Lensing cuts based on Mandelbaum 2017 (arxiv 1705.06745)

lensing_cuts = [
    ~GCRQuery((np.isnan, 'mag_i_cModel')), # (from this and below) remove nan entries
    ~GCRQuery((np.isnan, 'ext_shapeHSM_HsmShapeRegauss_resolution')),
    ~GCRQuery((np.isnan, 'ext_shapeHSM_HsmShapeRegauss_e1')),
    ~GCRQuery((np.isnan, 'ext_shapeHSM_HsmShapeRegauss_e2')),
    GCRQuery('good'), 
    GCRQuery('snr_i_cModel >= 10'), # (from this and below) cut on object properties
    GCRQuery('HSM_res >= 0.3'),
    GCRQuery('HSM_ell < 2.0'),
    GCRQuery('ext_shapeHSM_HsmShapeRegauss_sigma <= 0.4'),
    GCRQuery('mag_i_cModel < 24.5'), # FIXME: Doesnt have exinction correction
    GCRQuery('blendedness < 10**(-0.375)'),
]

In [ ]:
tract_number = 4850
quantities = ['mag_i_cModel', 'snr_i_cModel', 'HSM_res', 'HSM_ell', 'psf_fwhm_i']
d = catalog.get_quantities(quantities, 
                           filters=lensing_cuts, 
                           native_filters=[(lambda x: x==tract_number, 'tract')])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8,8))
for ax_this, quantity in zip(ax.flat, quantities):
    ax_this.hist(d[quantity], 100, range=([0,100] if quantity=='snr_i_cModel' else None))
    ax_this.set_xlabel(quantity)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
ax.hist(d['psf_fwhm_i'], 100);
mean = d['psf_fwhm_i'].mean()
ax.axvline(mean, c='C1');
ax.text(mean*1.01, ax.get_ylim()[1]*0.95, 'mean = {:.4g}'.format(mean));
ax.set_xlabel('PSF FWHM [arcsec]');